In [2]:
# Step 1️⃣ Clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5

# Step 2️⃣ Change directory
%cd yolov5

# Step 3️⃣ Install requirements
!pip install -r requirements.txt


fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/deepak/yolov5


In [3]:
# Step 4️⃣ Import libraries
import torch

# Step 5️⃣ Load pre-trained YOLOv5s model (good for bottle detection)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # you can change 's' to 'm' for medium

# Step 6️⃣ Detect objects in your image
img_path = '/content/drive/MyDrive/Project/plastic bottle.jpg'  # 👈 give your test image path
results = model(img_path)

# Step 7️⃣ Show results
results.show()

# Step 8️⃣ Save cropped objects
results.save(save_dir='/content/detections')  # cropped images will be saved


Using cache found in /home/deepak/.cache/torch/hub/ultralytics_yolov5_master


ImportError: cannot import name 'tarfile' from 'backports' (/home/deepak/anaconda3/lib/python3.11/site-packages/backports/__init__.py)

In [4]:
!pip install tensorflow

In [5]:
from PIL import Image

# Get detection results
for det in results.xyxy[0]:  # detections per image
    xmin, ymin, xmax, ymax, conf, cls = det

    # Example: COCO bottle class index = 39
    if int(cls.item()) == 39:
        im = Image.open(img_path)

        # ✅ Convert Tensor → int
        x1, y1, x2, y2 = int(xmin.item()), int(ymin.item()), int(xmax.item()), int(ymax.item())

        # ✅ Crop with plain Python ints
        crop = im.crop((x1, y1, x2, y2))

        # ✅ Save cropped image
        crop.save('/content/cropped_bottle.jpg')

        print("✅ Cropped image saved!")


NameError: name 'results' is not defined

In [6]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np



In [7]:
data_dir = '/home/deepak/Downloads/dataset/w/RealWaste/'

 # 3️⃣ DEFINE IMAGE SIZE + PATHS

IMG_SIZE = (240, 240)
BATCH_SIZE = 32

In [8]:
# 4️⃣ SPLIT & LOAD DATASET

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # 20% for validation, 80% for training
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [9]:
# 4️⃣ SPLIT & LOAD DATASET

train_ds = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'    # 👈 TRAINING
)

Found 3803 images belonging to 9 classes.


In [10]:

# 4️⃣ SPLIT & LOAD DATASET
val_ds = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # 👈 VALIDATION
)

Found 949 images belonging to 9 classes.


In [11]:

# 5️⃣ BUILD MODEL

base_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(train_ds.num_classes, activation='softmax')   # Use your class count
])

2025-07-06 06:20:40.138448: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [13]:
# 6️⃣ COMPILE MODE

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

# 7️⃣ TRAIN MODEL
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

/home/deepak/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 868s 7s/step - accuracy: 0.5790 - loss: 1.2041 - val_accuracy: 0.0917 - val_loss: 7.4580
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 767s 6s/step - accuracy: 0.8126 - loss: 0.5475 - val_accuracy: 0.0917 - val_loss: 3.9368
Epoch 3/20
 17/119 ━━━━━━━━━━━━━━━━━━━━ 6:02 4s/step - accuracy: 0.8886 - loss: 0.3611

In [ ]:
import matplotlib.pyplot as plt

# Accuracy plot
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
# Loss plot
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Load cropped image
img = load_img('/content/cropped_bottle.jpg', target_size=(240, 240))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)
predicted_class = np.argmax(pred)
print(f"Predicted class: {predicted_class} with {np.max(pred)*100:.2f}% confidence")


In [ ]:
class_labels = ['Cardboard', 'Food Organics', 'Glass', 'Metal',
                'Miscellaneous Trash', 'Paper', 'Plastic',
                'Textile Trash', 'Vegetation']

print("Final prediction:", class_labels[predicted_class])


In [ ]:

# 8️⃣ SAVE MODEL WEIGHTS
model.save('/content/drive/MyDrive/Project/efficientnetb1_waste.h5')

print("✅ Model training done & weights saved!")